<a href="https://colab.research.google.com/github/JeanMusenga/ASSORT-Automatic-Summarization-of-Stack-Overflow-Posts/blob/main/LSA_TextSummarizationWith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Important resources:
https://chatgpt.com/share/0fde4435-eb1d-4ab1-9149-78c0899251e0

# Importing necessary libraries

In [ ]:
pip install contractions

In [ ]:
pip install chardet

In [ ]:
import re
import string
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import TruncatedSVD
import contractions
import unicodedata
import chardet
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


In [ ]:
dataset=pd.read_excel('ArchIssueSolutionBench.xlsx')

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# converting all object types in string
dataset['Question_body'] = dataset['Question_body'].astype(str)
dataset['Answer_body'] = dataset['Answer_body'].astype(str)

# Preprocessing function

In [ ]:
# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Sentence tokenization
    sentences = sent_tokenize(text)

    # Stopwords
    stop_words = set(ENGLISH_STOP_WORDS)

    # Define punctuation and special characters
    punctuation = set(string.punctuation)
    extra_special_characters = set(["''", '``', '##', '>>', '<<', 'e', 'g', 'eg', 'cant', 'cannot', 'isnt', 'would', 'could', 'doesnt', 'hasnt', 'thanks', '-', ')', '\n'])
    special_characters = extra_special_characters.union(punctuation)

    processed_sentences = []

    for sentence in sentences:
        # Tokenization and cleaning
        cleaned_words = re.findall(r'\b\w+\b', sentence.lower())
        cleaned_words = [word for word in cleaned_words if word not in stop_words]
        cleaned_words = [word for word in cleaned_words if word not in special_characters]
        cleaned_words = [word for word in cleaned_words if not word.isdigit()]
        cleaned_words = [word for word in cleaned_words if word.isalpha()]

        # Lemmatization
        lemmatized_words = [lemmatizer.lemmatize(word) for word in cleaned_words]

        processed_sentences.append(lemmatized_words)

    return sentences, processed_sentences

In [ ]:
# Preprocess only the processed_sentences part
dataset['processed_question'] = dataset['Question_body_cleaned'].apply(lambda x: preprocess_text(x)[1])
dataset['processed_answer'] = dataset['Answer_body_cleaned'].apply(lambda x: preprocess_text(x)[1])

# Display the first five rows
dataset[['processed_question', 'processed_answer']].head()


,processed_question,processed_answer
0,"[[need, help, architecture, pattern, use, nest...","[[talking, strictly, cqrs, pattern, designing,..."
1,"[[file, structure, process, fromjson, tojson],...","[[purpose, clean, architecture, fromjson, tojs..."
2,"[[building, c, net, mvc, web, application, all...","[[easy, reliable, way, splitting, task, multip..."
3,"[[mobile, application, scale, white, label, de...","[[simple, answer, question], [think, tailwind,..."
4,"[[m, trying, properly, design, application, ac...","[[determining, source, information, business, ..."


# Build term-document matrix function

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

def build_similarity_matrix(processed_sentences):
    # Join words in each sentence
    sentences = [" ".join(sentence) for sentence in processed_sentences]

    # Compute the TF-IDF matrix
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)

    # Determine the number of components based on the number of features
    num_features = tfidf_matrix.shape[1]
    n_components = min(num_features, 100)  # Set to min(num_features, 100)

    # Apply LSA
    lsa = TruncatedSVD(n_components=n_components)
    lsa_matrix = lsa.fit_transform(tfidf_matrix)

    # Compute the cosine similarity matrix based on the LSA-transformed matrix
    similarity_matrix = cosine_similarity(lsa_matrix)

    return similarity_matrix

# Build similarity matrices for both processed questions and answers
dataset['question_similarity_matrix'] = dataset['processed_question'].apply(lambda x: build_similarity_matrix(x))
dataset['answer_similarity_matrix'] = dataset['processed_answer'].apply(lambda x: build_similarity_matrix(x))

dataset[['question_similarity_matrix', 'answer_similarity_matrix']].head()


# Define LSA function

In [ ]:
# Apply LSA
def apply_lsa(tfidf_matrix, num_topics=1):
    lsa = TruncatedSVD(n_components=num_topics, n_iter=100)
    lsa_matrix = lsa.fit_transform(tfidf_matrix)
    return lsa, lsa_matrix

# Define Ranking sentences function

In [ ]:
# Rank sentences
def rank_sentences(lsa_matrix):
    scores = np.mean(lsa_matrix, axis=1)
    return scores

# Define Function for Generating summary

In [ ]:
# Generate summary function
def generate_summary(original_sentences, scores, num_sentences=5):
    ranked_sentence_indices = np.argsort(scores)[::-1]
    summary = []
    for i in range(min(num_sentences, len(original_sentences))):
        summary.append(original_sentences[ranked_sentence_indices[i]])
    return " ".join(summary)

# Preprocess the Question_body and Answer_body columns

In [ ]:
# Preprocess the Question_body and Answer_body columns
dataset['processed_question'] = dataset['Question_body'].apply(preprocess_text)
dataset['processed_answer'] = dataset['Answer_body'].apply(preprocess_text)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def build_tfidf_matrix(sentences):
    # Convert list of tokenized sentences back to strings
    text_sentences = [" ".join(sentence) for sentence in sentences]

    # Create TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(text_sentences)

    return tfidf_matrix

from sklearn.decomposition import TruncatedSVD

def apply_lsa(tfidf_matrix, n_components=100):
    # Ensure the number of components is less than the number of features
    if n_components > tfidf_matrix.shape[1]:
        n_components = tfidf_matrix.shape[1]

    # Apply LSA (Latent Semantic Analysis)
    lsa = TruncatedSVD(n_components=n_components)
    lsa_matrix = lsa.fit_transform(tfidf_matrix)

    return lsa, lsa_matrix

# Apply term-document matrix building
dataset['question_tfidf_matrix'] = dataset['processed_question'].apply(lambda x: build_tfidf_matrix(x[1]))
dataset['answer_tfidf_matrix'] = dataset['processed_answer'].apply(lambda x: build_tfidf_matrix(x[1]))

# Apply LSA to the term-document matrices for both questions and answers
dataset['question_lsa'], dataset['question_lsa_matrix'] = zip(*dataset['question_tfidf_matrix'].apply(apply_lsa))
dataset['answer_lsa'], dataset['answer_lsa_matrix'] = zip(*dataset['answer_tfidf_matrix'].apply(apply_lsa))

# Preview the results
print(dataset[['question_lsa_matrix', 'answer_lsa_matrix']].head())


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


                                 question_lsa_matrix  \
0  [[0.0726994913694369, 0.308309941007228, 0.063...   
1  [[1.0326437600711701e-16, 9.13152076552896e-17...   
2  [[0.49639837347347116, 0.4849522575809426, -0....   
3  [[0.4992662958766415, -0.014905045990007752, 0...   
4  [[0.2445105082448705, -0.43545245727144394, 0....   

                                   answer_lsa_matrix  
0  [[0.1580528588587956, 0.02337172910397252, -0....  
1  [[0.7688152550772703, -0.2569437107197592, 0.1...  
2  [[0.12362497420677634, 0.1036725531259288, 0.3...  
3  [[-4.412360298992142e-16, 1.1306069957173599e-...  
4  [[0.5497160433297623, 0.3629888288657769, -0.3...  


# Apply LSA to the term-document matrices for both questions and answers

In [ ]:
# Rank sentences
dataset['question_scores'] = dataset['question_lsa_matrix'].apply(rank_sentences)
dataset['answer_scores'] = dataset['answer_lsa_matrix'].apply(rank_sentences)


# Generate summaries for both questions and answers

In [ ]:
# Generate summaries for both questions and answers
dataset['Issue_Extracted'] = dataset.apply(lambda x: generate_summary(x['processed_question'][0], x['question_scores']), axis=1)
dataset['Solution_Extracted'] = dataset.apply(lambda x: generate_summary(x['processed_answer'][0], x['answer_scores']), axis=1)


In [ ]:
# Display the summaries
summaries = dataset[['Issue_Extracted', 'Solution_Extracted']]
summaries.head()

,Issue_Extracted,Solution_Extracted
0,"I am still able to do stuff I need to do, I am...","mongoose), and define your relations well, you..."
1,I created entity class on business layer and a...,Entities defined in the business layer should ...
2,Should I set up a separate WCF service to hand...,There is no easy and reliable way to do this ...
3,"Application functionality may differ slightly,...",Think of how <strong>Tailwind</strong> or <str...
4,RemoteData - loads and caches data from the AP...,This &quot;feels&quot; wrong as it seems like ...


In [ ]:
# Normalize unicode characters
def normalize_unicode(text):
    return unicodedata.normalize('NFKD', text)

# Apply normalize_unicode to each element in the specified columns
dataset['Issue_Extracted'] = dataset['Issue_Extracted'].apply(normalize_unicode)
dataset['Solution_Extracted'] = dataset['Solution_Extracted'].apply(normalize_unicode)

In [ ]:
# Apply contractions.fix to each element in the specified columns
dataset['Issue_Extracted'] = dataset['Issue_Extracted'].apply(lambda x: contractions.fix(x))
dataset['Solution_Extracted'] = dataset['Solution_Extracted'].apply(lambda x: contractions.fix(x))

In [ ]:
# Function to replace problematic characters
def replace_problematic_characters(text):
    replacements = {
        'â€™': "'",
        'â€œ': '"',
        'â€': '"',
        'â€TM': "'",
        'â€¦': " ",


        # Add more replacements if necessary
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

In [ ]:
# Apply replace_problematic_characters to each element in the specified columns
dataset['Issue_Extracted'] = dataset['Issue_Extracted'].apply(replace_problematic_characters)
dataset['Solution_Extracted'] = dataset['Solution_Extracted'].apply(replace_problematic_characters)

In [ ]:
# Display the summaries
summaries = dataset[['Question_title', 'Issue_Extracted', 'Solution_Extracted']]
summaries.head()

,Question_title,Issue_Extracted,Solution_Extracted
0,Separation of Students and Users in NestJS Mic...,"I am still able to do stuff I need to do, I am...","mongoose), and define your relations well, you..."
1,Flutter Clean Architecture,I created entity class on business layer and a...,Entities defined in the business layer should ...
2,Correct .NET Architecture for long running asy...,Should I set up a separate WCF service to hand...,There is no easy and reliable way to do this ...
3,Architecture for white-label mobile apps with ...,"Application functionality may differ slightly,...",Think of how <strong>Tailwind</strong> or <str...
4,Implementing Data Source Selection Logic in Cl...,RemoteData - loads and caches data from the AP...,This &quot;feels&quot; wrong as it seems like ...


In [ ]:
# Save the summarized data to a new Excel file
file_path = '/content/LSA_Summaries.xlsx'

dataset.to_excel(file_path, index=False, engine='openpyxl')

In [ ]:
# Verify that the file has been saved correctly
saved_data = pd.read_excel(file_path)
print(saved_data[['Question_title', 'Issue_Extracted', 'Solution_Extracted']].head())

                                      Question_title  \
0  Separation of Students and Users in NestJS Mic...   
1                         Flutter Clean Architecture   
2  Correct .NET Architecture for long running asy...   
3  Architecture for white-label mobile apps with ...   
4  Implementing Data Source Selection Logic in Cl...   

                                     Issue_Extracted  \
0  I am still able to do stuff I need to do, I am...   
1  I created entity class on business layer and a...   
2  Should I set up a separate WCF service to hand...   
3  Application functionality may differ slightly,...   
4  RemoteData - loads and caches data from the AP...   

                                  Solution_Extracted  
0  mongoose), and define your relations well, you...  
1  Entities defined in the business layer should ...  
2   There is no easy and reliable way to do this ...  
3  Think of how <strong>Tailwind</strong> or <str...  
4  This &quot;feels&quot; wrong as it seems like ..